In [1]:
import requests # 用于请求
from bs4 import BeautifulSoup as BS # 用于找到东西
from time import sleep # 用于放慢速度
from fake_useragent import UserAgent # 用于生成随机UserAgent
import re #用于正则表达式
import json #用于json解析
import re #正则表达式

# wanna get the url

In [22]:
num = 0
url = f"https://feed.mix.sina.com.cn/api/roll/get?pageid=372&lid=2431&k=&num=50&page={num}&r=0.3823521415001323&callback=jQuery111207971611566913344_1693192765363&_=1693192765366"
user_agent = UserAgent()
random_user_agent = user_agent.random
headers_1 = {
    "User-Agent": random_user_agent,
}


In [30]:
url_list = set()
flag = 0
num = 1
while len(url_list)<6000 and num<1000:
    num = num+1
    url = f"https://feed.mix.sina.com.cn/api/roll/get?pageid=372&lid=2431&k=&num=50&page={num}&r=0.3823521415001323&callback=jQuery111207971611566913344_1693192765363&_=1693192765366"
    response=requests.get(url,headers_1)
    if response.status_code == 200: 
        patterns = r'"url":"(https:\\\/\\\/[^"]+\.shtml)"'
        matches = re.findall(patterns, response.text)
        for match in matches:
            match=match.replace("\\","")
            url_list.add(match)
    else:
        print("there is wrong")
        break
    if flag == len(url_list):
        print("没有爬取到新东西")
        break
    else:
        flag = len(url_list)
        print(flag)

print(len(url_list))

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
没有爬取到新东西
2450


In [141]:
def get_url(num:int):
    flag = 0
    time = 1693065600 + 86400
    day = 28
    month = 8
    page = 1
    url_list_2 = set()
    while len(url_list_2)<num:
        day = day - 1
        time = time - 86400
        if day == 0:
            month = month - 1
            if month ==7 or month == 5 or month ==3 or month ==1:
                day = 31
            elif month == 6 or month ==4:
                day =30
            elif month ==2:
                day = 28
            else:
                print("2023一年的过完了")
                break
        page = 0
        print(f"正在爬2023-0{month}-{day}的消息")
        while True:
            page = page+1
            url_data = f"https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2515&etime={time - 86400}&stime={time}&ctime={time}&date=2023-0{month}-{day}&k=&num=50&page=1&r=0.28122078933743455&callback=jQuery1112020761369332467994_1693206645292&_=1693206645293"
            print(url_data)
            response=requests.get(url_data,headers_1)
            if response.status_code == 200: 
                patterns = r'"url":"(https:\\\/\\\/finance.sina.com.cn\\\/tech\\\/[^"]+\.shtml)"'
                matches = re.findall(patterns, response.text)
                for match in matches:
                    match=match.replace("\\","")
                    url_list_2.add(match)
            else:
                print("there is wrong")
                break
            if flag == len(url_list_2):
                print("没有爬取到新东西")
                break
            else:
                flag = len(url_list_2)
                print(flag)
    return url_list_2            
url =  get_url(6000)   
with open('url_tech.txt', 'w') as file:
    for item in url:
        file.write(str(item) + '\n')   


正在爬2023-08-27的消息
https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2515&etime=1692979200&stime=1693065600&ctime=1693065600&date=2023-08-27&k=&num=50&page=1&r=0.28122078933743455&callback=jQuery1112020761369332467994_1693206645292&_=1693206645293
没有爬取到新东西
正在爬2023-08-26的消息
https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2515&etime=1692892800&stime=1692979200&ctime=1692979200&date=2023-08-26&k=&num=50&page=1&r=0.28122078933743455&callback=jQuery1112020761369332467994_1693206645292&_=1693206645293
45
https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2515&etime=1692892800&stime=1692979200&ctime=1692979200&date=2023-08-26&k=&num=50&page=1&r=0.28122078933743455&callback=jQuery1112020761369332467994_1693206645292&_=1693206645293
没有爬取到新东西
正在爬2023-08-25的消息
https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2515&etime=1692806400&stime=1692892800&ctime=1692892800&date=2023-08-25&k=&num=50&page=1&r=0.28122078933743455&callback=jQuery1112020761369332467994_1693206645292&

aim to get the useful thing in html

In [138]:
article_templete ={
        "title":"",
        "news_website": "",
        "author": "",
        "Managing Editor":"",
        "details": {
            "creation_time": "创建时间",
            "comments_count": "评论数量",
            "keywords":[]
        },
        "content": [],
        "urls": {
            "webpage": "网页URL",
            "image_url": []
        }
    }

article_list = []

In [139]:
# image_url
def find_photo_url(element, results):
    if element.name == 'div' and 'class' in element.attrs and "img_wrapper" in element['class']: #有img_wrapper
        for child in element.children:
            if child.name == 'img':
                results.append(child['src'])

    for child in element.children:
        if child.name is not None:
            find_photo_url(child, results)
            

# keywords
def find_keyword(element, results):
    if element.name == 'div' and 'class' in element.attrs and "keywords" in element['class']: #有img_wrapper
        for child in element.children:
            if child.name == 'a':
                results.append(child.get_text())

    for child in element.children:
        if child.name is not None:
            find_keyword(child, results)
            
# content
def find_body(element):
    if element.name == 'div' and 'id' in element.attrs and "artibody" in element['id']:
        return element    

    for child in element.children:
        if child.name is not None:
            result = find_body(child)
            if result is not None:
                return result
            
            
def find_content(element, results):
    if (element.name == 'p' or element.name == 'strong') and len(element.find_all()) == 0 : 
        str = element.get_text().replace("\u3000","")
        str.replace("\n", "").replace("\t", "").strip()
        results.append(str)
        
    for child in element.children:
        if child.name is not None:              
            find_content(child, results) 
            
def find_time(element, results):
    if element.name == 'span' and 'class' in element.attrs and "date" in element['class']: #有img_wrapper
            results.append(element.get_text())

    for child in element.children:
        if child.name is not None:
            find_time(child, results)

In [140]:
article = article_templete.copy()
url = "https://finance.sina.com.cn/tech/discovery/2023-08-24/doc-imzihuqy2308890.shtml"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BS(response.text, "lxml")

# web url
article["urls"]["webpage"] = url

# title
titles = soup.find_all('title')
if titles:
    title = titles[0].string.lstrip().split("_")
    if len(title) >1:
        article["title"]=title[0].replace("|","")
        article["news_website"] =title[1].replace("|","")
    else:
        article["title"]=title[0]
else:
    print(f"there is no title in the article from this url:{url},maybe something wrong")
   
# image_url
photo_urls = []
find_photo_url(soup,photo_urls)
for photo_url in photo_urls:
    article["urls"]["image_url"].append("https:"+photo_url)
 
# content and Managing Editor
contents = []
body = find_body(soup)
if body:
    find_content(body,contents)
    for content in contents:
        if "责任编辑：" in content:
            editor_name = content.replace("责任编辑"," ").strip()
            article["Managing Editor"] = editor_name
            contents.remove(content)
        if "文 |" in content:
            author_name = content.replace("文 |"," ").strip()
            article["author"] = author_name.replace("\xa0","")
            contents.remove(content)    
    article["content"] = contents
else:
    print("cannot find the body") 

#time
times = []
find_time(soup,times)
if times:
    article["details"]["creation_time"] = times[0]
else:
    print("there is no time")

# keywords
keywords = []
find_keyword(soup,keywords)
article["details"]["keywords"] = keywords

print(article)

{'title': '日本东电负责人称启动核污水排海是“重大里程碑”：24小时运行 排30年核污水日本东电核污染', 'news_website': '新浪科技', 'author': '', 'Managing Editor': '：建嘉', 'details': {'creation_time': '2023年08月24日 16:56', 'comments_count': '评论数量', 'keywords': ['核污水', '日本东电', '核污染', '福岛', '里程碑']}, 'content': ['快科技8月24日消息，今天中午12点，日本福岛第一核电站的核污染水排海工作正式启动。', '据东电介绍，今天的核污染水排放量预计为200到210吨，每天的排放情况将在次日公布。', '根据东电计算，用海水稀释过的核污染水将缓慢流过约1公里的隧道，约1000秒之后抵达大海。', '虽然日本和东电都一再强调，排放的水已经经过处理，甚至宣称对人体无害，但多国专家都强调，核污染水中的很多放射性元素根本无法清除，会影响全人类。', '据悉，我们日常生活中的环境辐射一般在0.05微西弗左右，国际上建议普通人的安全环境辐射值为每小时最高不超过0.2微西弗，现在测量仪显示的数值是1.70，也就是说在这片区域里，人体所承受的辐射量是标准值的8至9倍。', '【本文结束】如需转载请务必注明出处：快科技'], 'urls': {'webpage': 'https://finance.sina.com.cn/tech/discovery/2023-08-24/doc-imzihuqy2308890.shtml', 'image_url': ['https://n.sinaimg.cn/spider20230824/199/w600h399/20230824/c6de-855a0996d865406896c7b420eea6cfdb.jpg', 'https://n.sinaimg.cn/spider20230824/137/w600h337/20230824/5c23-f58fe4bee072953c995cca1a41fd94fa.png']}}
